# Anomaly Detection with UMAP 

### 📌사용할 데이터셋 
Kaggle TPS September Competition data : 고객이 보험에 가입했는지 여부를 예측하기 위한 데이터셋이다. <br/>
target값인 "claim"은 binary이고 (T/F) 예측값은 0에서 1사이의 값으로 나타날 수 있다. 



### 📌베이스라인 AD모델 
Isolation Forest : 밀도기반 이상탐지 모델로, 데이터셋을 의사결정나무로 형태로 표현해 정상값을 분리하기 위해서 의사결정나무 깊숙하게 타고 내려가야 하고, 반대로 이상값은 의사결정나무 상단부에서 분리할 수 있다는 것을 이용하는 모델이다. 

👍장점
* 군집기반 이상탐지 알고리즘에 비해 계산량이 매우 적다
* 강건한 모델을 만들 수 있다. 

#### 데이터 차원 축소(UMAP)적용

In [14]:
import datatable as dt
import pandas as pd
import umap

# Train data 읽어들이기(빠르게 읽어들여서 테이블 형식으로 만들기 위해 datatable의 fread 사용)
tps = dt.fread("C:/Users/JiwonMoon/Desktop/Data/tabular-playground-series-sep-2021/train.csv").to_pandas()

# 결측 삭제해줌 
tps = tps.dropna(axis=0)
tps.shape

(359464, 120)

In [15]:
# X,y 설정해주기
X,y = tps.drop("claim",axis=1), tps[["claim"]].values.flatten()

In [16]:
# Manifold 초기화 해줌 
#차원을 2차원으로 축소시킬거기 때문에 n_component는 2로 설정
manifold = umap.UMAP(n_components=2)
manifold.fit(X,y)

X_2d  = manifold.transform(X)

#### 베이스라인 모델구축 (Isolation Forest)

##### 🎯 벤치마크 모델 = catboost classifier
: 베이스라인 모델 구축하기 전에, 벤치마크 모델에 적용했을 때의 성능확인 

In [18]:
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer #결측치 채우는 패키지 
import catboost as cb
from sklearn.metrics import roc_auc_score 

# 벤치마크 모델 설정중 (Catboost)
tps = dt.fread("C:/Users/JiwonMoon/Desktop/Data/tabular-playground-series-sep-2021/train.csv").to_pandas()
X, y = tps.drop("claim", axis=1), tps[["claim"]].values.flatten()


X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, random_state=11212128, test_size=0.1
)

pipe = make_pipeline(
    SimpleImputer(),
    cb.CatBoostClassifier(task_type="GPU", n_estimators=1000, verbose=False),
)

pipe.fit(X_train, y_train)
preds = pipe.predict_proba(X_valid)

roc_auc_score(y_valid, preds[:, 1])

c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


0.7846295472199392

##### 베이스라인 모델에 적용 (Isolation Forest)

In [20]:
%%time
from sklearn.preprocessing import QuantileTransformer

tps = dt.fread("C:/Users/JiwonMoon/Desktop/Data/tabular-playground-series-sep-2021/train.csv").to_pandas()
X, y = tps.drop("claim", axis=1), tps[["claim"]].values.flatten()

# 파이프라인 구성(결측값 채우기, QuantileTransformer)
# Quantiletransform이란? : 1000개의 quantile을 이용해서 데이터를 균등하게 분포시킨다. quantile을 이용하는 Robust Scaler와 똑같이 outlier에 민감하지 않다. 전체데이터를 MinMaxScaler와 비슷하게 0과 1사이로 압축시킨다. 
pipe = make_pipeline(SimpleImputer(), QuantileTransformer(),umap.UMAP(n_components=5))
X_processed = pipe.fit_transform(X) 

# Isolation Forest 적용하고 정상(1) 비정상(-1) 예측 
iso = IsolationForest(n_estimators=500, n_jobs=9)
labels = iso.fit_predict(X_processed)

# 비정상이라고 구분된 label 개수 count하기 
import numpy as np 
print(np.sum(labels == -1)) 

c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


156917
CPU times: total: 2h 31min 25s
Wall time: 11min 26s


##### Outlier 제거 후 벤치마크 모델에 적용해서 성능 비교 

In [21]:
outlier_idx = np.where(labels == -1)[0]
X_reduced = X.copy().drop(outlier_idx)
y_reduced = np.delete(y, outlier_idx)

# Impute again
X_reduced_imputed = SimpleImputer().fit_transform(X_reduced)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_reduced_imputed, y_reduced, random_state=1121218, test_size=0.1
)

clf = cb.CatBoostClassifier(task_type="GPU", n_estimators=1000, verbose=False).fit(
    X_train, y_train
)

preds = clf.predict_proba(X_valid)
roc_auc_score(y_valid, preds[:, 1]) 

c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\JiwonMoon\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


0.7864196432357515

🔎 결과 해석 : <br/>
Isolation Forest를 이용해 outlier를 제거한 후 catboost를 사용했을 때의 AUC는 0.786으로 제거하지 않았을 때 성능에 비해 소폭상승함을 확인할 수 있다. 

#### Reference
https://towardsdatascience.com/how-to-use-umap-for-much-faster-and-effective-outlier-detection-e4608f336915 